In [3]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-129844")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-129844
Azure region: southcentralus
Subscription id: f39cb977-6a3a-445b-a26a-b9a791c5fd89
Resource group: aml-quickstarts-129844


In [5]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [45]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling( {
        'C': uniform(0.05, 1),
        'max_iter': choice(50, 100, 200)
    }
)

# Specify a Policy
policy = BanditPolicy(slack_factor=0.15, evaluation_interval=1, delay_evaluation=10)

if "training" not in os.listdir():
    os.mkdir("./training")
    
import shutil

shutil.copy('train.py', "training")

'training/train.py'

In [46]:
# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory="training",
             entry_script='train.py',
             compute_target=cpu_cluster)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                     hyperparameter_sampling=ps, 
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=6,
                                     max_concurrent_runs=4)

In [47]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
run = exp.submit(config=hyperdrive_config)

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


In [48]:
RunDetails(run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [ ]:
run.wait_for_completion(show_output=True)

RunId: HD_3080b626-b197-4bc2-a76a-a74537514722
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_3080b626-b197-4bc2-a76a-a74537514722?wsid=/subscriptions/f39cb977-6a3a-445b-a26a-b9a791c5fd89/resourcegroups/aml-quickstarts-129844/workspaces/quick-starts-ws-129844

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-12-08T20:53:38.748052][API][INFO]Experiment created<END>\n""<START>[2020-12-08T20:53:39.310569][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2020-12-08T20:53:39.515050][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2020-12-08T20:53:39.6050343Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>


In [20]:
import joblib
# Get your best run and save the model from that run.

best_run = run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])

model = best_run.register_model(model_name='sklearn-log-reg', model_path='outputs/model.joblib')

AttributeError: 'NoneType' object has no attribute 'get_details'

In [26]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

web_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=web_path)

In [44]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='regression',
    primary_metric='Accuracy',
    X=x,
    y=y,
    n_cross_validations=5)

In [2]:
# Submit your automl run

automl_run = exp.submit(config=automl_config)

In [ ]:
# Retrieve and save your best automl model.

import joblib
# Get your best run and save the model from that run.

best_automl_run = automl_run.get_best_run_by_primary_metric()
print(best_automl_run.get_details()['runDefinition']['arguments'])

automl_model = best_automl_run.register_model(model_name='automl_model', model_path='outputs/automl_model.joblib')